In [ ]:
import os
import pandas as pd
import xlrd

data = pd.read_excel('empresas.xlsx')
empresas = data.to_dict(orient="list")
fundamentos = {}
arquivos = os.listdir("balancos")
for arquivo in arquivos:
    nome = arquivo[-9:-4]
    if "11" in arquivo:
        nome = arquivo[-10:-4] 
    print(nome, end=',')
       
    if nome in empresas['IBOV']:
        # pegar o balanco da empresa sheet 0
        wb = xlrd.open_workbook(f'balancos/{arquivo}', logfile=open(os.devnull, 'w'))
        balanco = pd.read_excel(wb, engine='xlrd',sheet_name=0)
        
        # na primeira coluna colocar o titulo com o nome da empresa
        balanco.iloc[0,0] = nome
        
        # Pegar 1ª linha e tornar um cabecalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]
        
        # retornar a 1ª coluna
        balanco = balanco.set_index(nome)
        
        # Pegar o dre da empresa sheet 1        
        dre = pd.read_excel(f'balancos/{arquivo}', sheet_name=1)
         # na primeira coluna colocar o titulo com o nome da empresa
        dre.iloc[0,0] = nome
        
        # Pegar 1ª linha e tornar um cabecalho
        dre.columns = dre.iloc[0]
        dre = dre[1:]
        
        # retornar a 1ª coluna
        dre = dre.set_index(nome)
        
        # cria tabela fundamentos com o balanco e o dre juntos
        fundamentos[nome]= pd.concat([balanco,dre])

display(fundamentos["ABEV3"])   

In [ ]:
cotacoes_df = pd.read_excel('cotacoes.xlsx')
cotacoes  = {}
for empresa in cotacoes_df["IBOV"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['IBOV']==empresa, :]



In [ ]:
display(cotacoes["ABEV3"])

In [ ]:
for empresa in empresas['IBOV']:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)
empresas = list(cotacoes.keys())


In [ ]:
cp_fundamentos = fundamentos
    

In [ ]:
for empresa in fundamentos:
    tabela = fundamentos[empresa].T
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y")
    tabela_cotacao = cotacoes[empresa].set_index("Date")
    tabela_cotacao = tabela_cotacao[["Adj Close"]]
    
    tabela = pd.merge(tabela, tabela_cotacao,how='left', left_index=True, right_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela

display(fundamentos["ABEV3"])    

In [ ]:
display(fundamentos)

In [ ]:
colunas = list(fundamentos["ABEV3"].columns)

for empresa in empresas:
    if set(colunas) != set(fundamentos[empresa].columns):
        fundamentos.pop(empresa)


In [ ]:
texto_colunas = ";".join(colunas)

colunas_modificadas = []
for coluna in colunas:
    if colunas.count(coluna) == 2 and coluna not in colunas_modificadas:
        texto_colunas = texto_colunas.replace(";" + coluna + ";",";" + coluna + "_1;", 1)
        colunas_modificadas.append(coluna)
colunas = texto_colunas.split(";")


In [ ]:
# implementar as colunas nas tabelas
for empresa in fundamentos:
    fundamentos[empresa].columns  = colunas

display(fundamentos["ABEV3"])  

In [ ]:
# analisar valores vazios nas colunas

valores_vazios = dict.fromkeys(colunas,0)

total_linhas = 0
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()        
        valores_vazios[coluna] += qtde_vazios

print(valores_vazios)



In [ ]:
remover_colunas = []
for coluna in valores_vazios:
    if valores_vazios[coluna] > 300:
        remover_colunas.append(coluna)
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)
    fundamentos[empresa] = fundamentos[empresa].ffill()

display(fundamentos["ABEV3"])

In [631]:
data_inicial = "12/20/2012"
data_final = "12/31/2022"

from pandas_datareader import data as web
df_ibov = web.DataReader('^BVSP', data_source='yahoo', start=data_inicial, end=data_final)
display(df_ibov)


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-12-20,61276.0,60622.0,60992.0,61276.0,3083400.0,61276.0
2012-12-21,61271.0,60221.0,61271.0,61007.0,3157400.0,61007.0
2012-12-26,61322.0,60861.0,61003.0,60960.0,2598000.0,60960.0
2012-12-27,61191.0,60233.0,60964.0,60416.0,2852600.0,60416.0
2012-12-28,61066.0,60416.0,60416.0,60952.0,2626000.0,60952.0
...,...,...,...,...,...,...
2022-08-22,111487.0,109858.0,111487.0,110501.0,11010900.0,110501.0
2022-08-23,112965.0,110503.0,110504.0,112857.0,11892000.0,112857.0
2022-08-24,113888.0,112632.0,112856.0,112898.0,13758600.0,112898.0


In [ ]:
import numpy as np
datas  = fundamentos["ABEV3"].index
for data in datas:
    if data not in df_ibov.index:
        df_ibov.loc[data] = np.nan

df_ibov = df_ibov.sort_index()
df_ibov = df_ibov.ffill()
df_ibov = df_ibov.rename(columns={"Adj Close": "IBOV"})

for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].merge(df_ibov[["IBOV"]], left_index=True, right_index=True)




In [ ]:
print(list(fundamentos["ABEV3"].columns))
print(list(cp_fundamentos["ABEV3"].columns))

In [ ]:
# tornar indicadores em percentuais
# fundamentos%tri = fundamento tri / fundamento tri anterior

for empresa in fundamentos:
    fundamento = fundamentos[empresa]
    fundamento = fundamento.sort_index()
    for coluna in fundamento:
        if "Adj close" in coluna or "IBOV" in coluna:
            pass
        else:
            # pegar cotacao anterior
            condicoes = [
                (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
            ]
            valores = [
                -1,
                1,
                (abs(fundamento[coluna].shift(1) - abs(fundamento[coluna]) / abs(fundamento[coluna].shift(1)))),
                1,
                -1,
                1,
            ]
            fundamento[coluna] =np.select(condicoes, valores, default=fundamento[coluna] / fundamento[coluna].shift(1) - 1)
    fundamentos[empresa] = fundamento
    # pegar cotacao seguinte
    fundamento["Adj Close"] = fundamento["Adj Close"].shift(1) - fundamento["Adj Close"] - 1
    fundamento["IBOV"] = fundamento["IBOV"].shift(1) - fundamento["IBOV"] - 1
    fundamento["Resultado"] = fundamento["Adj Close"] - fundamento["IBOV"]
    condicoes = [
        (fundamento["Resultado"] > 0),
        (fundamento["Resultado"] < 0) & (fundamento["Resultado"] >= -0.02),
        (fundamento["Resultado"] < -0.02),
    ]
    valores = [2,1,0]
    fundamento["Decisao"] = np.select(condicoes, valores)
display(fundamentos["ABEV3"])

            

In [ ]:
# remover valores vazios
colunas = list(fundamentos["ABEV3"].columns)
valores_vazios = dict.fromkeys(colunas,0)
total_linhas = 0
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()       
        valores_vazios[coluna] += qtde_vazios
print(valores_vazios)
print(total_linhas)

In [ ]:
remover_colunas=[]
for coluna in valores_vazios:
    if valores_vazios[coluna] > (total_linhas / 3):
        remover_colunas.append(coluna)    

remover_colunas.append("Adj Close")
remover_colunas.append("IBOV")
remover_colunas.append("Resultado")

for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)
    fundamentos[empresa] = fundamentos[empresa].fillna(0)

print(fundamentos["ABEV3"].shape)


In [ ]:
copia_fundamentos = fundamentos.copy()

In [ ]:
base_dados = pd.DataFrame()
for empresa in copia_fundamentos:
    copia_fundamentos[empresa] = copia_fundamentos[empresa][1:-1] # pega da segunda linha até a penultima ( excliu a primeira e ulrima linha da tabela)
    copia_fundamentos[empresa] = copia_fundamentos[empresa].reset_index(drop=True)
    base_dados = pd.concat([base_dados,copia_fundamentos[empresa]])
    

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

display(base_dados["Decisao"].value_counts(normalize=True).map("{:.1%}".format))
fig = px.histogram(base_dados, x="Decisao", color="Decisao")
fig.show()


In [ ]:
correlacoes = base_dados.corr()

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(correlacoes, cmap="Wistia", ax=ax)
plt.show()


In [ ]:
display(correlacoes)

In [ ]:
correlacoes_encontradas = []
for coluna in correlacoes:
    for linha in correlacoes.index:
        if linha != coluna:
            valor = abs(correlacoes.loc[linha, coluna])
            if valor > 0.8 and (coluna,linha,valor) not in correlacoes_encontradas:
                correlacoes_encontradas.append((linha, coluna, valor))
                print(f"Correlação encontrada: {linha} e {coluna}. Valor: {valor}")
                

In [ ]:
remover = ["Ativo Circulante","Contas a Receber_1","Tributos a Recuperar","Passivo Total","Passivo Circulante","Passivo Não Circulante","Capital Social Realizado","Receita Líquida de Vendas e/ou Serviços","Lucro/Prejuízo do Período","Intangível"]
base_dados = base_dados.drop(remover, axis=1)

In [ ]:
print(base_dados.shape)

In [ ]:
# trinar arvore de decisao e pegar as caracteristicas mais importantes dela
from sklearn.ensemble import ExtraTreesClassifier

modelo = ExtraTreesClassifier(random_state=1)
x = base_dados.drop("Decisao", axis=1)
y = base_dados["Decisao"]
modelo.fit(x, y)

caracteristicas_importantes = pd.DataFrame(modelo.feature_importances_, x.columns).sort_values(by=0, ascending=False)

display(caracteristicas_importantes)
top10 = list(caracteristicas_importantes.index)[:10]
print(top10)


In [ ]:
# normalizar valores
from sklearn.preprocessing import StandardScaler

def ajustar_scaler(tabela_original):
    scaler = StandardScaler()
    tabela_auxiliar = tabela_original.drop("Decisao", axis=1)
    tabela_auxiliar = pd.DataFrame(scaler.fit_transform(tabela_auxiliar), tabela_auxiliar.index, tabela_auxiliar.columns )
    tabela_auxiliar["Decisao"]  = tabela_original["Decisao"]
    return tabela_auxiliar

nova_base_dados = ajustar_scaler(base_dados)
top10.append("Decisao")

nova_base_dados = nova_base_dados[top10].reset_index(drop=True)


In [ ]:
display(nova_base_dados)


In [ ]:
# separar dados de treino e dados de teste
from sklearn.model_selection import train_test_split

x = nova_base_dados.drop("Decisao", axis=1)
display(x)

In [ ]:
y = nova_base_dados["Decisao"]

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, random_state=1)


In [ ]:
# criar IA
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix

dummy = DummyClassifier(strategy="stratified", random_state=2)
dummy.fit(x_treino, y_treino)
previsao_dummy = dummy.predict(x_teste)


In [ ]:
def avaliar(y_teste, previsoes, nome_modelo):
    print(nome_modelo)
    report = classification_report(y_teste, previsoes)
    print(report)
    cf_matrix = pd.DataFrame(confusion_matrix(y_teste, previsoes), index=["Vender", "Comprar"], columns=["Vender", "Comprar"])
    sns.heatmap(cf_matrix, annot=True, cmap="Blues", fmt=",")
    plt.show()
    print("*" * 50)

avaliar(y_teste, previsao_dummy, "Dummy")



In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

modelos = {
    "AdaBoost": AdaBoostClassifier(random_state=1),
    "DecisonTree": DecisionTreeClassifier(random_state=1),
    "RandomForest": RandomForestClassifier(random_state=1),
    "ExtraTree": ExtraTreesClassifier(random_state=1),
    "GradientBoost": GradientBoostingClassifier(random_state=1),
    "KNN": KNeighborsClassifier(),
    "LogisticRegression": LogisticRegression(random_state=1),
    "NaiveBayes": GaussianNB(),
    "SVM": SVC(random_state=1),
    "RedeNeural": MLPClassifier(random_state=1, max_iter=400),    
}

In [ ]:
for nome_modelo in modelos:
    modelo = modelos[nome_modelo]
    modelo.fit(x_treino, y_treino)
    previsoes = modelo.predict(x_teste)
    avaliar(y_teste, previsoes, nome_modelo)
    modelos[nome_modelo] = modelo
    

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score

modelo_final = modelos["RandomForest"]

n_estimators = range(10, 251, 30)
max_features = list(range(2, 11, 2))
max_features.append('sqrt')
min_samples_split = range(2, 11, 2)

precision2_score = make_scorer(precision_score, labels=[2], average='macro')

grid = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={
        'n_estimators': n_estimators,
        'max_features': max_features,
        'min_samples_split': min_samples_split,
        'random_state': [1],
    },
    scoring=precision2_score,
)

resultado_grid = grid.fit(x_treino, y_treino)
print('Ajuste feito')


In [ ]:
modelo_tunado = resultado_grid.best_estimator_
previsoes = modelo_tunado.predict(x_teste)
avaliar(y_teste, previsoes, "RandomForest Tunado")

In [ ]:
ult_tri_fundamentos = fundamentos.copy()
ult_tri_base_dados = pd.DataFrame()
lista_empresas = []

for empresa in ult_tri_fundamentos:
    ult_tri_fundamentos[empresa] = ult_tri_fundamentos[empresa][-1:] # pega ultima linha
    ult_tri_fundamentos[empresa] = ult_tri_fundamentos[empresa].reset_index(drop=True)
    ult_tri_base_dados =  pd.concat([ult_tri_base_dados, ult_tri_fundamentos[empresa]])
    lista_empresas.append(empresa)



In [639]:
display(ult_tri_fundamentos['ABEV3'])
display(ult_tri_base_dados)
print(lista_empresas)

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber_1,Estoques_1,Tributos a Recuperar,Outros Ativos Circulantes,Ativo Realizável a Longo Prazo,Tributos Diferidos_1,...,Resultado da Equivalência Patrimonial,Financeiras,Receitas Financeiras,Despesas Financeiras,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período,Decisao
0,-0.014641,-0.096069,-0.210989,0.178183,0.035174,0.035968,-0.053483,0.464569,0.338638,0.0,...,11002.856935,1.0,4.80323,1.514254e+06,2.481764,1.0,-1.0,84168.239952,1.983122,0


,Investimentos,Outros Ativos Não Circulantes,Tributos Diferidos_1,Ativo Total,Obrigações Fiscais,Ativo Realizável a Longo Prazo,Fornecedores,Empréstimos e Financiamentos,Resultado Antes Tributação/Participações,Resultado da Equivalência Patrimonial
0,0.406249,0.193903,-0.277350,-0.500141,0.822740,2.441089,0.469060,-0.536558,2.073792,-0.090816
1,0.617157,1.164855,-0.277350,-1.265463,-0.500927,0.750124,-0.718716,3.507656,-0.739087,-0.294221
2,-2.856027,0.193903,3.605551,-0.770003,-0.011190,-0.439975,0.042115,-0.192390,0.183725,-0.294221
3,-0.393254,0.193903,-0.277350,1.755012,1.086546,-0.188497,0.079059,-0.180919,0.936979,-0.294221
4,0.515074,0.193903,-0.277350,0.033961,-1.238059,-0.451190,-0.586453,-0.036937,-0.989924,-0.294218
5,0.725704,0.193903,-0.277350,-0.406181,-1.092760,-0.955048,-0.081290,-0.284571,0.570112,-0.294207
6,0.474121,0.193903,-0.277350,-0.482770,0.370198,-0.368136,-1.287369,-0.188373,-1.355569,-0.294220
7,0.431225,0.193903,-0.277350,0.840389,-0.385641,0.644394,0.718659,-0.281402,0.750521,-0.294196
8,-1.746890,0.193903,-0.277350,-0.246221,-0.315816,-0.504828,-0.794711,-0.052424,0.578274,-0.294240
9,0.519819,0.193903,-0.277350,2.098261,1.314837,0.627194,1.499451,0.177504,-0.705525,-0.294233


['ABEV3', 'BBSE3', 'BRFS3', 'CIEL3', 'CMIG4', 'CSAN3', 'ELET3', 'ENBR3', 'GOLL4', 'MGLU3', 'MULT3', 'PETR4', 'PRIO3', 'VALE3']


In [ ]:
ult_tri_base_dados = ult_tri_base_dados.reset_index(drop=True)
ult_tri_base_dados = ult_tri_base_dados[top10]
ult_tri_base_dados = ajustar_scaler(ult_tri_base_dados)
ult_tri_base_dados = ult_tri_base_dados.drop("Decisao", axis=1)

display(ult_tri_base_dados)

In [636]:
previsoes_ult_tri = modelo_tunado.predict(ult_tri_base_dados)
print(previsoes_ult_tri)

carteira = []
carteira_inical = []

for i, empresa in enumerate(lista_empresas):
    if previsoes_ult_tri[i] == 2:
        print(empresa)
        carteira_inical.append(1000)
        cotacao = cotacoes[empresa]
        cotacao = cotacao.set_index("Date")
        cotacao_inicial = cotacao.loc["2020-12-01", "Adj Close"]
        cotacao_final = cotacao.loc["2021-03-31", "Adj Close"]
        percentual = cotacao_final / cotacao_inicial
        carteira.append(1000 * percentual)

saldo_inicial = sum(carteira_inical)
saldo_final = sum(carteira)
print(saldo_inicial, saldo_final)
print(saldo_final / saldo_inicial)


[2 0 2 0 0 0 0 0 0 0 2 0 2 0]
ABEV3
BRFS3
MULT3
PRIO3
4000 5004.2846253809275
1.251071156345232


In [ ]:
print(df_ibov)

In [635]:
variacao_ibov = df_ibov.loc["2021-03-31", "IBOV"] / df_ibov.loc["2020-12-01","IBOV"]
print(variacao_ibov)



1.047595095881798


In [ ]:
# armazenar a inteligencia arificial
# import joblib 

# joblib.dump(modelo_tunado,"ia_carteiraasacoes.joblib")

# modelo_tunado = joblib.load("ia_carteiraasacoes.joblib")
